# Restaurant atmosphere/vibes analysis

I found some apps, that contain images of restaurants along with a vibe labelled.

`Tripadvisor` only has `business meetings`, `romantic`, and ideal for `groups vibes`. So it is not enough.

After searching a looot of apps, I found the `One Zone app`. Desgined specifically to chose a restaurant by the vibe they evoke. However, the dataset is really small, as they add restaurants manually. They don't use any kind of deep learning method. Maybe we can use it as a test dataset.

Therefore, what I am going to try to do, is to develop a NLP model to classify customer reviews by the restaurant vibe. Then I would label the restaurants images by the vibe obtained with the NLP model.

Yelp Dataset

So we'll be working with Yelp dataset. The Yelp Open Dataset is an extensive collection provided by Yelp for academic and research purposes. It offers a rich source of data from Yelp's vast repository of local business reviews and user interactions. It contains:

1. **Businesses:**
Basic information about local businesses, including their name, location (address, city, state, postal code), latitude and longitude, average rating, category (e.g., Restaurants, Shopping, Beauty & Spas), and the number of reviews they have received. This allows for analyses focused on geographical trends, category-based studies, and more.

2. **Reviews:**
Millions of text reviews written by Yelp users for various businesses. Each review includes the user's text review, the star rating they gave, and the date of the review. This data is invaluable for sentiment analysis, natural language processing tasks, and understanding consumer preferences.

3. **Users:**
Information about Yelp users who have written reviews, including their user ID, name, review count, yelping since (the date they joined Yelp), friends, and other social metrics. This can be used for social network analysis, studying user behavior, and personalization algorithms.

4. **Check-ins:**
Data about check-ins at businesses by Yelp users, which can help in analyzing foot traffic trends, popular times, and loyalty or frequency of visits.

5. **Tips:**
Short notes left by users about a business. Tips can contain advice, recommendations, or comments about what's good or what to avoid. This dataset is useful for extracting quick insights or highlights about businesses.

6. **Photos:**
Metadata about photos associated with businesses, including a photo ID, business ID, caption, and labels indicating the photo category (e.g., food, interior, exterior). This can support visual analysis of businesses or complement review-based insights as in our case.

## CSV File prepared

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# EDA

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/restaurant_reviews_with_photos.csv")

<ipython-input-3-9d5eea62fb33>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/restaurant_reviews_with_photos.csv")


In [4]:
df.head()

,business_id,text,photo_id
0,XQfwVwDr-v0ZS3_CbbE5Xw,"If you decide to eat here, just be aware it is...",NaN
1,YjUWPpI6HXG530lwP-fb2A,Family diner. Had the buffet. Eclectic assortm...,NaN
2,kxX2SOes4o-D3ZQBkiMRfA,"Wow! Yummy, different, delicious. Our favo...",NaN
3,e4Vwtrqf-wpJfwesgvdgxQ,Cute interior and owner (?) gave us tour of up...,-rCqVHSxxfNSCBLvFE_U6Q
4,04UD14gamNjLY0IDYVhHJg,I am a long term frequent customer of this est...,1eKNwiFMPTLfviad0Sh-Ew


In [5]:
df.shape

(4724684, 3)

In [6]:
df['photo_id'].isnull().sum()

4709621

In [7]:
df = df.dropna()

In [8]:
df.shape

(15063, 3)

In [9]:
df.head()

,business_id,text,photo_id
3,e4Vwtrqf-wpJfwesgvdgxQ,Cute interior and owner (?) gave us tour of up...,-rCqVHSxxfNSCBLvFE_U6Q
4,04UD14gamNjLY0IDYVhHJg,I am a long term frequent customer of this est...,1eKNwiFMPTLfviad0Sh-Ew
6,gebiRewfieSdtt17PTW6Zg,Had a party of 6 here for hibachi. Our waitres...,NML2x1GKH-ZrDLMWgljdTw
7,EQ-TZ2eeD_E0BHuvoaeG5Q,"Locals recommended Milktooth, and it's an amaz...",6ZCkhDPDjoNCi0khcHIJxw
8,lj-E32x9_FA7GmUrBGBEWg,Love going here for happy hour or dinner! Gre...,bEXUuGQrQftJyMoYFGlwog


In [10]:
df['text']

3          Cute interior and owner (?) gave us tour of up...
4          I am a long term frequent customer of this est...
6          Had a party of 6 here for hibachi. Our waitres...
7          Locals recommended Milktooth, and it's an amaz...
8          Love going here for happy hour or dinner!  Gre...
                                 ...                        
4644837    This restaurant is tucked away on a hill and i...
4646338    It's absolutely Delicious! All the food was co...
4647271    I went in thinking I was gonna order cupcakes ...
4653065    Let me start by saying how much I love the par...
4665892    The food is not bad at all, but the prices are...
Name: text, Length: 15063, dtype: object

In [11]:
df.shape

(15063, 3)

In [12]:
df['business_id'].duplicated().sum()

0

In [13]:
from transformers import pipeline

# Load a sentiment analysis pipeline
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Example text
text = "The ambiance of the restaurant was cozy and inviting, with dim lighting and soft music."

# Predict the sentiment
result = classifier(text)

print(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': '4 stars', 'score': 0.3810330629348755}]


In [15]:
df['text'].iloc[6]

"Tremendous service (Big shout out to Douglas) that complemented the delicious food. Pretty expensive establishment (40-50$ avg for your main course), but its definitely backs that up with an atmosphere that's comparable with any of the top tier restaurants across the country."

In [16]:
df['text'].iloc[8]

"I was really between 3 and 4 stars for this one. I LOVE the 96th street Naked Tchopstix so I was very excited to see this one which is closer to my house.\nThe vibe is totally different as this is geared more to take out although they do have a decent size dining area. You order at the counter and they deliver it.\nMy daughter and I tried the sushi bowl. You pick up a piece of paper and  select from a choice of proteins, vegetables, rice and toppings. I like the fact that it is on paper and you don't have to guess what things are or have to shout over the counter to tell the server what you want. The sushi bowl was good but not wonderful. The choices particularly of the vegetables seemed a little too limiting and I felt I had more rice than other food items. The sauces were tasty. My husband ordered the Maui roll which is my favorite roll at the 96th street location and this just was not up that standard. It was good but the quality and consistently was not as good as at 96th street. 

In [17]:
df['text'].iloc[9]

'Went for lunch. Beef brisket sandwich was awesome.   So juicy and tender.    Pulled pork was was just as good!'

In [21]:
df['photo_id'].iloc[9]

'7ZBZG-O2PZ3HrJaHguGEYQ'

In [18]:
df['text'].iloc[10]

'Best thai food in the area.  Everything was authentic and delicious.  Will definitely be back again and again.'

In [19]:
df['text'].iloc[15]

'What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.'

In [20]:
df['photo_id'].iloc[15]

'sRHnMbvjCIhxs_Yss33ENQ'